# <center><h1>Decision Tree Implementation for Iris Dataset

### <div style="text-align: right"> -Utkarsh Maheshwari </div>

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math

In [13]:
class DecTree:
    def __init__(self):
        self.__root = None

    def __count_unique(self,Y):
#         returns unique values as keys with their count as value
        d = {}
        for i in Y:
            if i not in d:
                d[i]=1
            else:
                d[i]+=1
        return d


    def __entropy(self,Y):
#         return entropy
        freq_map = self.__count_unique(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
        return entropy_

    def __gain_ratio(self,X,Y,selected_feature):
#          return gain ratio
        info_orig = self.__entropy(Y) # ig before split
        info_f = 0  # ig after split
        split_info = 0
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)

#         to avoid /0 error
        if split_info == 0 :
            return math.inf

        info_gain = info_orig - info_f
        gain_ratio = info_gain / split_info
        return gain_ratio

    def __gini_index(self,Y):
#         return gini index
        freq_map = self.__count_unique(Y)
        gini_index_ = 1
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            gini_index_ -= p**2
        return gini_index_

    def __gini_gain(self,X,Y,selected_feature):
#         returns gini gain
    
        
        gini_orig = self.__gini_index(Y) # gi before split
        gini_split_f = 0 # gi after split
        values = set(X[:,selected_feature])
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y
        initial_size = df.shape[0] 
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            gini_split_f += (current_size/initial_size)*self.__gini_index(df1[df1.shape[1]-1])

        gini_gain_ = gini_orig - gini_split_f
        return gini_gain_


    def __decision_tree(self,X,Y,features,level,metric,classes):
#          returns the root of the Decision Tree
#          classes represents the different classes 
#          level represents depth of the tree
#          once we splpit a node on a feature we remove it from available features
        
        
#       STOPPING CRITERIA
#      1) PURE NODE
#      2) NO FEATURE LEFT
        
        # If the node consists of only 1 class
        if len(set(Y)) == 1:
            print("Level",level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
                    print("Count of",i,"=",len(Y))
                else :
                    print("Count of",i,"=",0)
            if metric == "gain_ratio":
                print("Current Entropy is =  0.0")
            elif metric == "gini_index":
                print("Current Gini Index is =  0.0")

            print("Reached leaf Node")
            print()
            return MyNode(None,output)

        # no avaliable features
        if len(features) == 0:
            print("Level",level)
            freq_map = self.__count_unique(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    print("Count of",i,"=",0)
                else :
                    if freq_map[i] > max_count :
                        output = i
                        max_count = freq_map[i]
                    print("Count of",i,"=",freq_map[i])

            if metric == "gain_ratio":
                print("Current Entropy  is =",self.__entropy(Y))
            elif metric == "gini_index":
                print("Current Gini Index is =",self.__gini_index(Y))            

            print("Reached leaf Node")
            print()
            return MyNode(None,output)

        
        # best feature to split upon
        max_gain = -math.inf
        final_feature = None
        for f in features :
            if metric == "gain_ratio":
                current_gain = self.__gain_ratio(X,Y,f)
            elif metric =="gini_index":
                current_gain = self.__gini_gain(X,Y,f)

            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f

        print("Level",level)
        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf

        for i in classes:
            if i not in freq_map:
                print("Count of",i,"=",0)
            else :
                if freq_map[i] > max_count :
                    output = i
                    max_count = freq_map[i]
                print("Count of",i,"=",freq_map[i])

        if metric == "gain_ratio" :        
            print("Current Entropy is =",self.__entropy(Y))
            print("Splitting on feature  X[",final_feature,"] with gain ratio ",max_gain,sep="")
            print()
        elif metric == "gini_index":
            print("Current Gini Index is =",self.__gini_index(Y))
            print("Splitting on feature  X[",final_feature,"] with gini gain ",max_gain,sep="")
            print()

            
        unique_values = set(X[:,final_feature]) 
        df = pd.DataFrame(X)
        df[df.shape[1]] = Y

        current_node = MyNode(final_feature,output)

#         Now removing the selected feature from the list 
        index  = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
#              Creating a new dataframe with value of selected feature = i
            df1 = df[df[final_feature] == i]
#             Segregating the X and Y values and recursively calling on the splits
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:,df1.shape[1]-1].values,features,level+1,metric,classes)
            current_node.add_child(i,node)

#          Add the removed feature     
        features.insert(index,final_feature)

        return current_node
    
    def fit(self,X,Y,metric="gain_ratio"):
#          Fits to the given training data
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
#         if metric != "gain_ratio" :
#             if metric != "gini_index":
#                 metric="gain_ratio"  
        self.__root = self.__decision_tree(X,Y,features,level,metric,classes)
        
    def __predict_for(self,data,node):
        # predicts the class for given test case
        
#         we have reached leaf node
        if len(node.children) == 0 :
            return node.output

        val = data[node.data]        
        if val not in node.children :
            return node.output
        
#      Recursion call
        return self.__predict_for(data,node.children[val])

    def predict(self,X):
#         return predictions
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i],self.__root)
        return Y
    
    def score(self,X,Y):
#          returns the mean accuracy
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count+=1
        return count/len(Y_pred)
    
    def export_tree_pdf(self,filename=None):
#       returns the tree as dot data
       
        import pydotplus
        from collections import deque
        
        dot_data = '''digraph Tree {
node [shape=box] ;'''
        
        queue = deque()
        
        r = self.__root
        queue.append(r)
        count = 0
        if r.index == -1:
            r.index = count
        
        dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(count,r.data,r.output) 
        
  
        while len(queue) != 0 :
            node = queue.popleft()
            for i in node.children:
                count+=1
                if(node.children[i].index==-1):
                    node.children[i].index = count
                
                # Creating child node
                dot_data = dot_data + "\n{} [label=\"Feature to split upon : X[{}]\\nOutput at this node : {}\" ];".format(node.children[i].index,node.children[i].data,node.children[i].output) 
                # Connecting parent node with child
                dot_data = dot_data + "\n{} -> {} [ headlabel=\"Feature value = {}\"]; ".format(node.index,node.children[i].index,i)
                # Adding child node to queue
                queue.append(node.children[i])
        
        dot_data = dot_data + "\n}"

        if filename != None:    
            graph = pydotplus.graph_from_dot_data(dot_data)
            graph.write_pdf(filename)    
        
        return dot_data

In [14]:

class MyNode:
    def __init__(self, data,output):
#         data = feature to split on
        self.data = data
#      children are stored n from a dict   
        self.children = {}
        # output is prediction class
        self.output = output
#         to get unique index
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

### data preperation

In [15]:
# loading data
iris = datasets.load_iris()
# split data 
X, xtest ,Y , ytest =  train_test_split(iris.data , iris.target ,random_state = 0)
# convert continuous valued data to categorical data
mean_columns = []
for k in range(X.shape[-1]):
    mean_columns.append(X[:,k].mean())
for i in range(len(X)):
    for j in range(len(X[0])):
        val = mean_columns[j]
        if(X[i][j]>val):
            X[i][j] = 1
        else:
            X[i][j] = 0

### Displaying Values for every label

In [16]:
# creating a object of DecisionTreeClassifier class 
clf2 = DecTree()

In [17]:
# call fit function
clf2.fit(X,Y,metric='gini_index')

Level 0
Count of 0 = 37
Count of 1 = 34
Count of 2 = 41
Current Gini Index is = 0.6647002551020409
Splitting on feature  X[2] with gini gain 0.2754061147967782

Level 1
Count of 0 = 37
Count of 1 = 6
Count of 2 = 0
Current Gini Index is = 0.24012979989183347
Splitting on feature  X[1] with gini gain 0.10059491617090324

Level 2
Count of 0 = 6
Count of 1 = 6
Count of 2 = 0
Current Gini Index is = 0.5
Splitting on feature  X[3] with gini gain 0.045454545454545414

Level 3
Count of 0 = 6
Count of 1 = 5
Count of 2 = 0
Current Gini Index is = 0.4958677685950414
Splitting on feature  X[0] with gini gain 0.0

Level 4
Count of 0 = 6
Count of 1 = 5
Count of 2 = 0
Current Gini Index is = 0.4958677685950414
Reached leaf Node

Level 3
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 2
Count of 0 = 31
Count of 1 = 0
Count of 2 = 0
Current Gini Index is =  0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 28
Count of 2 = 41
Current Gini Inde

### prediction

In [18]:
# call predict function 
print("--------------------------------------------------------------")
Y_pred2 = clf2.predict(X)
print("Predictions->",Y_pred2)

--------------------------------------------------------------
Predictions-> [2 1 2 0 2 0 0 0 2 2 2 2 1 2 2 1 2 2 1 2 2 2 2 0 1 1 1 1 2 2 0 0 2 1 0 0 2
 0 2 0 0 2 2 1 0 2 2 2 2 0 0 2 2 0 2 0 1 2 0 0 2 0 0 0 1 2 2 0 0 0 1 2 0 0
 2 0 2 0 2 2 0 2 0 2 0 0 2 0 2 1 1 0 2 2 2 2 0 0 2 1 0 2 1 2 2 0 0 0 2 1 2
 0]


### Displaying Tree

In [20]:
print("tree in dot data fromat",clf2.export_tree_pdf(filename="iris_tree.pdf"))

tree in dot data fromat digraph Tree {
node [shape=box] ;
0 [label="Feature to split upon : X[2]\nOutput at this node : 2" ];
1 [label="Feature to split upon : X[1]\nOutput at this node : 0" ];
0 -> 1 [ headlabel="Feature value = 0.0"]; 
2 [label="Feature to split upon : X[0]\nOutput at this node : 2" ];
0 -> 2 [ headlabel="Feature value = 1.0"]; 
3 [label="Feature to split upon : X[3]\nOutput at this node : 0" ];
1 -> 3 [ headlabel="Feature value = 0.0"]; 
4 [label="Feature to split upon : X[None]\nOutput at this node : 0" ];
1 -> 4 [ headlabel="Feature value = 1.0"]; 
5 [label="Feature to split upon : X[3]\nOutput at this node : 1" ];
2 -> 5 [ headlabel="Feature value = 0.0"]; 
6 [label="Feature to split upon : X[1]\nOutput at this node : 2" ];
2 -> 6 [ headlabel="Feature value = 1.0"]; 
7 [label="Feature to split upon : X[0]\nOutput at this node : 0" ];
3 -> 7 [ headlabel="Feature value = 0.0"]; 
8 [label="Feature to split upon : X[None]\nOutput at this node : 1" ];
3 -> 8 [ headlab